In [1]:
import numpy as np
from mpl_toolkits import mplot3d
import open3d as o3d
import glob
import os
import os.path
import statistics as stats

In [2]:
# Variables:

slice_size = .02
clustering_eps = 6
min_points = 200
percent_ground_cutoff = .97

#Flags
#write out slice, ground, or just final plant

In [19]:
# Full Pass
plant = r"C:\Users\Travis\OneDrive - College of Coastal Georgia\work\3D vis\3D_ply_files_small\3D_ply_files_small\c105084f-77c2-4d84-9514-20d0b1e28b95__Top-heading-east_0.ply"

# ------------------------------------------------------
# Slicing out a portion

# Read .ply file
pcd_whole = o3d.io.read_point_cloud(plant)

# Convert open3d format to numpy array
# Here, you have the point cloud in numpy format. 
ar_whole_field = np.asarray(pcd_whole.points)

# Slicing
max_1 = max(ar_whole_field[:,1])
min_1 = min(ar_whole_field[:,1])

length = [max_1,min_1]
yrange = (abs(max_1)-abs(min_1))*.02
middle = stats.median(length)
bound_1 = middle + yrange
bound_2 = middle - yrange

print(f'max = {max_1}\nmin = {min_1}\nMiddle = {middle}\nYrange = {yrange}')
slice_of_whole = []

for index, row in enumerate(ar_whole_field):
    if bound_1 > bound_2:
        if bound_1 > row[1] >  bound_2:
            slice_of_whole.append(row)

    if bound_2 > bound_1:
        if bound_2 > row[1] >  bound_1:
            slice_of_whole.append(row)
            
            
pcd_slice = o3d.geometry.PointCloud()
pcd_slice.points = o3d.utility.Vector3dVector(slice_of_whole)

ar_slice = np.asarray(pcd_slice.points)

o3d.io.write_point_cloud(f"C:\\Users\\Travis\\OneDrive - College of Coastal Georgia\\work\\Untitled Folder\\slice.ply", pcd_slice)

#------------------------------------------------
# Sending the slice to be clustered

# Clustering
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(pcd_slice.cluster_dbscan(eps=6, min_points=200, print_progress=True))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

l0 = []

# Separating out the labels
for index, row in enumerate(ar_slice):
    lab = labels[index]
    if lab == 0:
        l0.append(row)

#---------------------------------
# Getting the extent of the ground to crop out the plants

pcd_ground = o3d.geometry.PointCloud()
pcd_ground.points = o3d.utility.Vector3dVector(l0)
ar_ground = np.asarray(pcd_ground.points) 
# Extents for cropping
max_0 = max(ar_ground[:,0])
max_2 = max(ar_ground[:,2])
min_0 = min(ar_ground[:,0])

o3d.io.write_point_cloud(f"C:\\Users\\Travis\\OneDrive - College of Coastal Georgia\\work\\Untitled Folder\\ground.ply", pcd_ground)


#-----------------------------------------------------
# Cropping out the plants
plant = []
for i in ar_whole:
    if (min_0 < i[0] < max_0) and (i[2] > (max_2*.97)):
        plant.append(i)
pcd_plant = o3d.geometry.PointCloud()
pcd_plant.points = o3d.utility.Vector3dVector(plant)

o3d.io.write_point_cloud(f"C:\\Users\\Travis\\OneDrive - College of Coastal Georgia\\work\\Untitled Folder\\plants.ply", pcd_plant)



max = -3464.1064453125
min = -25659.125
Middle = -14561.61572265625
Yrange = -443.90037109375
[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 90
point cloud has 90 clusters


True